# Hugging Face transformers

La librería `transformers` de Hugging Face es una de las librerías más populares para trabajar con modelos de lenguaje. Su facilidad de uso hizo que se democratizara el uso de la arquitectura `Transformer` y que se pudiera trabajar con modelos de lenguaje de última generación sin necesidad de tener un gran conocimiento en el área.

Entre la librería `transformers`, el hub de modelos y su facilidad de uso, los spaces y la facilidad de desplegar demos y nuevas librerías como `datasets`, `accelerate`, `PEFT` y otras más, han hecho que Hugging Face sea uno de los actores más importantes de la escena de inteligencia artificial del momento. Ellos mismos se auto-denominan como "el GitHub de la IA" y ciertamente lo son.

## Instalación

Para instalar transformers se puede hacer con `pip`

```bash
pip install transformers
```

o con `conda`

```bash
conda install conda-forge::transformers
```

Además de la librería es necesario tener un backend de PyTorch o TensorFlow instalado. Es decir, necesitas tener instalar `torch` o `tensorflow` para poder usar `transformers`.

## Inferencia con `pipeline`

Con los `pipeline`s de `transformers` se puede hacer inferencia con modelos de lenguaje de una manera muy sencilla. Esto tiene la ventaja de que el desarrollo se realiza de manera mucho más rápida y se puede hacer prototipado de manera muy sencilla. Además permite a personas que no tienen mucho conocimiento poder usar los modelos

Con `pipeline` se puede hacer inferencia en un montón de tareas diferentes. Cada tarea tiene su propio `pipeline`, pero se puede hacer una abstracción general usando la clase `pipeline` que se encarga de seleccionar el `pipeline` adecuado para la tarea que se le pase.

### Tareas

Al día de escribir este post, las tareas que se pueden hacer con `pipeline` son:

 * Audio:
   * Clasificación de audio
     * clasificación de escena acústica: etiquetar audio con una etiqueta de escena (“oficina”, “playa”, “estadio”)
     * detección de eventos acústicos: etiquetar audio con una etiqueta de evento de sonido (“bocina de automóvil”, “llamada de ballena”, “cristal rompiéndose”)
     * etiquetado: etiquetar audio que contiene varios sonidos (canto de pájaros, identificación de altavoces en una reunión)
     * clasificación de música: etiquetar música con una etiqueta de género (“metal”, “hip-hop”, “country”)

 * Reconocimiento automático del habla (ASR, audio speech recognition):

 * Visión por computadora
   * Clasificación de imágenes
   * Detección de objetos
   * Segmentación de imágenes
   * Estimación de profundidad

 * Procesamiento del lenguaje natural (NLP, natural language processing)
   * Clasificación de texto
     * análisis de sentimientos
     * clasificación de contenido
   * Clasificación de tokens
     * reconocimiento de entidades nombradas (NER, por sus siglas en inglés): etiquetar un token según una categoría de entidad como organización, persona, ubicación o fecha.
     * etiquetado de partes del discurso (POS, por sus siglas en inglés): etiquetar un token según su parte del discurso, como sustantivo, verbo o adjetivo. POS es útil para ayudar a los sistemas de traducción a comprender cómo dos palabras idénticas son gramaticalmente diferentes (por ejemplo, “corte” como sustantivo versus “corte” como verbo)
   * Respuestas a preguntas
     * extractivas: dada una pregunta y algún contexto, la respuesta es un fragmento de texto del contexto que el modelo debe extraer
     * abstractivas: dada una pregunta y algún contexto, la respuesta se genera a partir del contexto; este enfoque lo maneja la Text2TextGenerationPipeline en lugar del QuestionAnsweringPipeline que se muestra a continuación
   * Resumir
     * extractiva: identifica y extrae las oraciones más importantes del texto original
     * abstractiva: genera el resumen objetivo (que puede incluir nuevas palabras no presentes en el documento de entrada) a partir del texto original
   * Traducción
   * Modelado de lenguaje
     * causal: el objetivo del modelo es predecir el próximo token en una secuencia, y los tokens futuros están enmascarados
     * enmascarado: el objetivo del modelo es predecir un token enmascarado en una secuencia con acceso completo a los tokens en la secuencia

 * Multimodal
   * Respuestas a preguntas de documentos

### Uso de `pipeline`

La forma más sencilla de crear un `pipeline` es simplemente indicarle la tarea que queremos que resuelva mediante el parámetro `task`. Y la librería se encargará de seleccionar el mejor modelo para esa tarea, descargarlo y guardarlo en la caché para futuros usos.

In [1]:
from transformers import pipeline

generator = pipeline(task="text-generation")

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [2]:
generator("Me encanta aprender de")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Me encanta aprender de se résistance davant que hiens que préclase que ses encasas quécénces. Se présentants cet en un croyne et cela désirez'}]

Como se puede ver el texto generado está en francés, mientras que yo se lo he introducido en español, por lo que es importante elegir bien el modelo.

Vamos a instanciar usar un modelo reentrenado en español mediante el parámetro `model`.

In [1]:
from transformers import pipeline

generator = pipeline(task="text-generation", model="flax-community/gpt-2-spanish")

In [2]:
generator("Me encanta aprender de")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Me encanta aprender de tus palabras, que con gran entusiasmo y con el mismo conocimiento como lo que tú acabas escribiendo, te deseo de todo corazón todo el deseo de este día:\nY aunque también haya personas a las que'}]

Ahora el texto generado tiene mucha mejor pinta

La clase `pipeline` tiene muchos posibles parámetros, por lo que para verlos todos y aprender más sobre la clase te recomiendo leer su [documentación](https://huggingface.co/docs/transformers/v4.38.1/en/main_classes/pipelines), pero vamos a hablar de una, ya que para el deep learning es muy importante y es `device`. Define el dispositivo (por ejemplo, `cpu`, `cuda:1`, `mps` o un rango ordinal de GPU como `1`) en el que se asignará el `pipeline`.

En mi caso, como tengo una GPU pongo `0`

In [11]:
from transformers import pipeline

generator = pipeline(task="text-generation", model="flax-community/gpt-2-spanish", device=0)

generation = generator("Me encanta aprender de")
print(generation[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Me encanta aprender de ustedes, a tal punto que he decidido escribir algunos de nuestros contenidos en este blog, el cual ha sido de gran utilidad para mí por varias razones, una de ellas, el trabajo


### Cómo funciona `pipeline`

Cuando hacemos uso de `pipeline` por debajo lo que está pasando es esto

![transformers-pipeline](http://maximofn.com/wp-content/uploads/2024/02/transformers-pipeline.svg)

Automáticamente se está tokenizando el texto, se pasa por el modelo y después por un postprocesado

## Inferencia con `AutoClass`

Hemos visto que `pipeline` nos abstrae mucho de lo que pasa, pero nosotros podemos seleccionar que tokenizador, que modelo y que postprocesado queremos usar.

### Tokenización con `AutoTokenizer`

Antes usamos el modelo `flax-community/gpt-2-spanish` para generar texto, podemos usar su tokenizador

In [1]:
from transformers import AutoTokenizer

checkpoint = "flax-community/gpt-2-spanish"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

text = "Me encanta lo que estoy aprendiendo"

tokens = tokenizer(text, return_tensors="pt")
print(tokens)

{'input_ids': tensor([[ 2879,  4835,   382,   288,  2383, 15257]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


### Modelo `AutoModel`

Ahora podemos crear el modelo y pasarle los tokens

In [21]:
from transformers import AutoModel

model = AutoModel.from_pretrained("flax-community/gpt-2-spanish")

output = model(**tokens)
type(output), output.keys()

(transformers.modeling_outputs.BaseModelOutputWithPastAndCrossAttentions,
 odict_keys(['last_hidden_state', 'past_key_values']))

Si ahora lo intentamos usar en un `pipeline` nos dará un error

In [23]:
from transformers import pipeline

pipeline("text-generation", model=model, tokenizer=tokenizer)("Me encanta aprender de")

The model 'GPT2Model' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'P

TypeError: The current model class (GPT2Model) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'GPT2LMHeadModel'}

Hesto es porque cuando funcionaba usábamos

```python
pipeline(task="text-generation", model="flax-community/gpt-2-spanish")
```

Pero ahora hemos hecho

```python
tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish")
model = AutoModel.from_pretrained("flax-community/gpt-2-spanish")
pipeline("text-generation", model=model, tokenizer=tokenizer)
```

En el primer caso solo usábamos `pipeline` y el nombre del modelo, por debajo buscaba la mejor manera de implementar el modelo y el tokenizador. Pero en el segundo caso hemos creado el tokenizador y el modelo y se lo hemos pasado a `pipeline`, pero no los hemos creado bien para lo que el `pipeline` necesita

Para arreglar esto usamos `AutoModelFor`

### Modelo `AutoModelFor`

La librería transformers nos da la oportunidad de crear un modelo para una tarea determinada como

 * `AutoModelForCausalLM` que sirve para continuar textos
 * `AutoModelForMaskedLM` que se usa para rellenar huecos
 * `AutoModelForMaskGeneration` que sirve para generar máscaras
 * `AutoModelForSeq2SeqLM` que se usa par convertir de secuencias a secuencias, como por ejemplo en traducción
 * `AutoModelForSequenceClassification` para clasificación de texto
 * `AutoModelForMultipleChoice` para elección múltiple
 * `AutoModelForNextSentencePrediction` para predecir si dos frases son consecutivas
 * `AutoModelForTokenClassification` para clasificación de tokens
 * `AutoModelForQuestionAnswering` para preguntas y respuestas
 * `AutoModelForTextEncoding` para codificación de texto

Vamos a usar el modelo anterior para generar texto

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish")
model = AutoModelForCausalLM.from_pretrained("flax-community/gpt-2-spanish")

pipeline("text-generation", model=model, tokenizer=tokenizer)("Me encanta aprender de")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Me encanta aprender de mi familia.\nLa verdad no sabía que se necesitaba tanto en este pequeño restaurante ya que mi novio en un principio había ido, pero hoy me ha entrado un gusanillo entre pecho y espalda que'

Ahora si funciona, porque hemos creado el modelo de una manera que `pipeline` puede entender

## Tokenización

Hemos visto que con `AutoTokenizer` se puede tokenizar el texto, pero hay cosas importantes de esta clase que contar

### Padding

Cuando se tiene un batch de secuencias, a veces es necesario que después de tokenizar, todas las secuencias tengan la misma longitu, así que para ello usamos el parámetro `padding=True`

In [3]:
from transformers import AutoTokenizer

batch_sentences = [
    "Pero, ¿qué pasa con el segundo desayuno?",
    "No creo que sepa lo del segundo desayuno, Pedro",
    "¿Qué hay de los elevensies?",
]

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish", pad_token="PAD")
encoded_input = tokenizer(batch_sentences, padding=True)
for encoded in encoded_input["input_ids"]:
    print(encoded)
print(f"Padding token id: {tokenizer.pad_token_id}")

[2959, 16, 875, 3736, 3028, 303, 291, 2200, 8080, 35, 50257, 50257]
[1489, 2275, 288, 12052, 382, 325, 2200, 8080, 16, 4319, 50257, 50257]
[1699, 2899, 707, 225, 72, 73, 314, 34630, 474, 515, 1259, 35]
Padding token id: 50257


Como vemos a las dos primeras secuencias les ha añadido un paddings al final

### Truncado

A parte de añadir padding, a veces es necesario truncar las secuencias para que no ocupen más de un número determinado de tokens. Para ello establecemos `truncation=True` y `max_length` con el número de tokens que queremos que tenga la secuencia

In [13]:
from transformers import AutoTokenizer

batch_sentences = [
    "Pero, ¿qué pasa con el segundo desayuno?",
    "No creo que sepa lo del segundo desayuno, Pedro",
    "¿Qué hay de los elevensies?",
]

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish")
encoded_input = tokenizer(batch_sentences, truncation=True, max_length=5)
for encoded in encoded_input["input_ids"]:
    print(encoded)

[2959, 16, 875, 3736, 3028]
[1489, 2275, 288, 12052, 382]
[1699, 2899, 707, 225, 72]


### Tensores

Hasta ahora estábamos recibiendo listas de tokens, pero seguramente nos interese recibir tensores de PyTorch o TensorFlow. Para ello usamos el parámetro `return_tensors` y le especificamos de qué framework queremos recibir el tensor, en nuestro caso elegiremos PyTorch con `pt`

In [14]:
from transformers import AutoTokenizer

batch_sentences = [
    "Pero, ¿qué pasa con el segundo desayuno?",
    "No creo que sepa lo del segundo desayuno, Pedro",
    "¿Qué hay de los elevensies?",
]

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish", pad_token="PAD")
encoded_input = tokenizer(batch_sentences, padding=True)
for encoded in encoded_input["input_ids"]:
    print(type(encoded))

<class 'list'>
<class 'list'>
<class 'list'>
Padding token id: 50257


Recibimos listas, si queremos recibir tensores de PyTorch usamos `return_tensors="pt"`

In [17]:
from transformers import AutoTokenizer

batch_sentences = [
    "Pero, ¿qué pasa con el segundo desayuno?",
    "No creo que sepa lo del segundo desayuno, Pedro",
    "¿Qué hay de los elevensies?",
]

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish", pad_token="PAD")
encoded_input = tokenizer(batch_sentences, padding=True, return_tensors="pt")
for encoded in encoded_input["input_ids"]:
    print(type(encoded), encoded.shape)
print(type(encoded_input["input_ids"]), encoded_input["input_ids"].shape)

<class 'torch.Tensor'> torch.Size([12])
<class 'torch.Tensor'> torch.Size([12])
<class 'torch.Tensor'> torch.Size([12])
<class 'torch.Tensor'> torch.Size([3, 12])


## Train

Hasta ahora hemos usado modelos preentrenados, pero en el caso que se quiera hacer fine tuning, la librería `transformers` lo deja muy fácil de hacer

Como hoy en día los modelos de lenguaje son enormes, reentrenarlos es casi imposible en una GPU que cualquiera pueda tener en su casa, por lo que vamos reentrenar un modelo más pequeño. En este caso vamos a reentrenar `bert-base-cased` que es un modelo de 109M parámetros.

### Dataset

Tenemos que descargarnos un dataset, para ello usamos la librería `datasets` de Hugging Face. Vamos a usar el conjunto de datos de reseñas de Yelp está formado por reseñas de Yelp.

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Vamos a ver que pinta tiene el dataset

In [2]:
type(dataset)

datasets.dataset_dict.DatasetDict

Parece que es una especie de diccionario, vamos a ver qué claves tiene

In [3]:
dataset.keys()

dict_keys(['train', 'test'])

Vamos a ver cuantas reseñas tiene en cada subconjunto

In [4]:
len(dataset["train"]), len(dataset["test"])

(650000, 50000)

Vamos a ver una muestra

In [5]:
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

Como vemos cada muestra tiene el texto y la puntuación, vamos a ver cuantas clases hay

In [6]:
dataset["train"].features

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None),
 'text': Value(dtype='string', id=None)}

Vemos que tiene 5 clases distintas

Vamos a ver una muestra de test

In [7]:
dataset["test"][100]

{'label': 0,
 'text': 'This was just bad pizza.  For the money I expect that the toppings will be cooked on the pizza.  The cheese and pepparoni were added after the crust came out.  Also the mushrooms were out of a can.  Do not waste money here.'}

### Tokenización

Ya tenemos el dataset, como hemos visto en el pipeline, primero se realiza la tokenización y después se aplica el modelo. Por lo que tenemos que tokenizar el dataset

Definimos el tokenizador

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

La clase `AutoTokenizer` tiene un método llamado `map` que nos permite aplicar una función al dataset, por lo que vamos a crear una función que tokenize el texto

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [10]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Vemos ejemplos del dataset tokenizado

In [11]:
tokenized_dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [12]:
tokenized_dataset["test"][100]

{'label': 0,
 'text': 'This was just bad pizza.  For the money I expect that the toppings will be cooked on the pizza.  The cheese and pepparoni were added after the crust came out.  Also the mushrooms were out of a can.  Do not waste money here.',
 'input_ids': [101,
  1188,
  1108,
  1198,
  2213,
  13473,
  119,
  1370,
  1103,
  1948,
  146,
  5363,
  1115,
  1103,
  27001,
  1116,
  1209,
  1129,
  13446,
  1113,
  1103,
  13473,
  119,
  1109,
  9553,
  1105,
  185,
  8043,
  17482,
  11153,
  1127,
  1896,
  1170,
  1103,
  21508,
  1338,
  1149,
  119,
  2907,
  1103,
  25590,
  1116,
  1127,
  1149,
  1104,
  170,
  1169,
  119,
  2091,
  1136,
  5671,
  1948,
  1303,
  119,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


Como vemos se ha añadido una key con los `token_ids` de los tokens, los `token_type_ids` y otra con la `atención mask`.

### Modelo

Tenemos que crear el modelo que vamos a reentrenar. Como es un problema de clasificación vamos a usar `AutoModelForSequenceClassification`

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Como se puede ver se ha creado un modelo que clasifica entre 5 clases

### Métrica de evaluación

Creamos una métrica de evaluación con la librería `evaluate` de Hugging Face. Para instalarla usamos

```bash
pip install evaluate
```

In [14]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### Trainer

Ahora para entrenar usamos el objeto `Trainer`. Para poder usar `Trainer` necesitamos `accelerate>=0.21.0`

```bash
pip install accelerate>=0.21.0
```

In [19]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

Una vez tenemos un `Trainer`, en que hemos indicado el dataset de entrenamiento, el de test, el modelo y la métrica de evaluación, podemos entrenar el modelo con el método `train` del `Trainer`

In [18]:
trainer.train()

  0%|          | 0/243750 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 3.82 GiB of which 77.94 MiB is free. Including non-PyTorch memory, this process has 3.73 GiB memory in use. Of the allocated memory 3.02 GiB is allocated by PyTorch, and 41.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)